# ANALYSE DES RÉSERVATIONS À LA MÉDIATHÈQUE

## Projet Tutoré 2025



* [x] Envoyer le mail type pour les réservations qui sont disponibles 



### VOS BESOINS STATS SUR LES RESAS
* [x] Quelle évolution du nombre de réservations ?
* [x] Quelle répartition des réservations effectuées par type de carte ?
* [x] Quel rythme des réservations (nb de réservations (par semaines et par année où la réservation a été effectuée) ?
* [x] ~~Ratio~~ Quelle évolution des réservations passées vs nb de résas expirées ou annulées ?
* [x] À quelle fréquence les adhérents réservent-ils ?
* [x] Nb de de réservations récupérées par un adhérent par type de public ?
* [x] Nombre de réservations par espaces ? de réserveurs distincts par espaces ?
* [x] Nombre de réservations passées par ancienneté du document ? (Les usagers préférent-ils les nouveautés) ?  
* [ ] ~~Les utilisateurs sont-il réactifs pour venir retirer leurs réservations~~ ?
* [ ] ~~Les utilisateurs sont-ils des réserveurs compulsifs (réservations groupées) ou réservent-ils peu mais de manière régulières ?~~
* [ ] ~~Rythme des retraits de réservations ?~~

**=> Objectif :** dresser une typologie des publics réservant des docs à la médiathèque

> ***Note :*** *Par réservations, on ne garde ici que celles effectuées par des adhérents inscrits sur le site Médiathèque. Sont donc exclues ici les réservations d'adhérents inscrits au bibliobus.*

### MAIL TYPE DE DISPONIBILITÉ D'UNE RÉSERVATION
> Bonjour <<borrowers.firstname>> <<borrowers.surname>>,  
> Vous avez réservé les ouvrages suivants : 

>Titre: <<biblio.title>>
>Auteur: <<biblio.author>>
>Cote : <<items.itemcallnumber>>
>Site: <<branches.branchname>>
>Date de mise de côté : <<reserves.waitingdate>>

>Ceux-ci sont désormais disponibles. Vous pouvez venir les retirer dès aujourd'hui et pendant les 7 jours à venir.

>Bonne journée, 
>L'équipe de La Grand-Plage

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

from kiblib.utils.db import DbConn
from kiblib.utils.code2libelle import Code2Libelle

import seaborn as sns

import locale
locale.setlocale(locale.LC_TIME,'')

'fr_FR.UTF-8'

In [2]:
info_vacances = pd.read_csv("/home/kibini/kibini2/kibini/kiblib/vacances_scolaires_fr.csv")

In [3]:
info_vacance_2024 = info_vacances[info_vacances['date'].astype('Datetime64').dt.year==2024]

In [4]:
info_vacance_2024_b = info_vacance_2024[['date','vacances_zone_b','nom_vacances']]

In [5]:
info_vacance_2024_b['date'] = pd.to_datetime(info_vacance_2024_b['date'])

/tmp/ipykernel_30783/3287252464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_vacance_2024_b['date'] = pd.to_datetime(info_vacance_2024_b['date'])


In [6]:
info_vacance_2024_b.dtypes

date               datetime64[ns]
vacances_zone_b              bool
nom_vacances               object
dtype: object

In [7]:
db_conn = DbConn().create_engine()

In [8]:
# Import des libs

c2l = Code2Libelle(db_conn=db_conn)
c2l.get_val()
c2l = c2l.dict_codes_lib

c2l_cardtype = c2l['carte']
df_types_cartes = pd.DataFrame.from_dict(c2l['carte']).T
df_types_cartes.head(3)

,carte,carte_type,category_type,carte_gratuite,carte_prix,carte_personnalite
BIBL,Médiathèque,Médiathèque,C,gratuite,0,Personne
CGEN,Compte générique,Gestion,I,gratuite,0,Gestion
CLAS,Collectivités (Classes maternelles et primaires),Service collectivités,I,gratuite,0,Collectivité


In [9]:
query = """SELECT *
FROM statdb.stat_reserves sr
WHERE YEAR(reservedate) > 2019"""

In [10]:
resas = pd.read_sql(query,db_conn)
resas

,reserve_id,borrowernumber,reservedate,biblionumber,branchcode,notificationdate,cancellationdate,priority,found,timestamp,...,categorycode,fidelite,motif_annulation,courriel,mobile,annulation,document_mis_cote,ccode,issuedate,waiting_duration
0,214698,54568,2020-01-01,312592.0,MED,None,None,0,W,2020-01-30 10:05:23,...,BIBL,1.0,None,oui,oui,non,oui,None,NaT,NaN
1,214699,10813,2020-01-01,322113.0,MED,None,2020-02-04,0,None,2020-02-04 08:54:38,...,MEDC,15.0,None,oui,non,oui,non,None,NaT,NaN
2,214700,1781,2020-01-01,291951.0,MED,None,None,0,W,2020-01-02 10:47:20,...,MEDA,12.0,None,oui,non,non,oui,None,NaT,NaN
3,214701,43164,2020-01-01,300678.0,MED,None,None,2,None,2020-01-02 03:19:03,...,BIBL,3.0,None,oui,oui,None,None,None,NaT,NaN
4,214702,43164,2020-01-01,310354.0,MED,None,None,1,None,2020-01-02 03:19:03,...,BIBL,3.0,None,oui,oui,None,None,None,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238599,482453,68918,2025-01-29,372371.0,BUS,None,None,1,None,2025-01-29 18:53:38,...,BIBL,3.0,None,oui,oui,None,None,None,NaT,NaN
238600,482454,68918,2025-01-29,225516.0,BUS,None,None,1,None,2025-01-29 18:54:53,...,BIBL,3.0,None,oui,oui,None,None,None,NaT,NaN
238601,482455,80801,2025-01-29,368753.0,MED,None,2025-01-29,0,None,2025-01-29 19:19:52,...,MEDP,1.0,None,oui,oui,None,None,None,NaT,NaN
238602,482456,37360,2025-01-29,347249.0,MED,None,None,1,None,2025-01-29 19:33:40,...,BIBL,9.0,None,oui,oui,None,None,None,NaT,NaN


In [11]:
resas['Nb de réservations effectuées'] = 1

In [12]:
#resas.loc[resas['annulation']=='oui',['Nb de réservations annulées']] = 1

In [13]:
resas['reservedate'] = pd.to_datetime(resas['reservedate'])

In [14]:
resas = resas.merge(info_vacance_2024_b,left_on='reservedate',right_on='date',how='left')

In [15]:
resas['reservedate'] = pd.to_datetime(resas['reservedate'],yearfirst=True)

In [16]:
resas['année de la réservation'] = resas['reservedate'].dt.year

resas['n° de semaine'] = resas['reservedate'].dt.isocalendar().week
resas['jour de la semaine'] = resas['issuedate'].dt.day_name(locale='fr_FR.utf8')

In [17]:
# On créer une requête qui extrait le contenu du champ 210$d contenant la date de publication
query2 = """SELECT biblionumber, ExtractValue(metadata, '//datafield[@tag="210"]/subfield[@code="d"]') AS "Date de publication"
FROM koha_prod.biblio_metadata"""

# On importe un dataframe contenant la date de publication et le biblionumber
bib_creationdate = pd.read_sql(query2,db_conn)

# On fait une jointure de l'info date de publication sur le dataframe resas
resas = resas.merge(bib_creationdate,left_on='biblionumber',right_on='biblionumber',how='left')

### Nettoyage du champ publicationdate

In [18]:
# On supprime tous les caractères qui ne sont pas des chiffres
resas['Date de publication'] = resas['Date de publication'].str.replace('\D','',regex=True)

# On suppose que le publicationdate commence toujours par l'année
# Donc on ne garde que les 4 premiers caractères
resas['Date de publication'] = resas['Date de publication'].str.slice(0,4)

# On remplit les cellules vides par l'info None
resas["Date de publication"] = resas["Date de publication"].fillna('None')

# On ne garde qui contiennent exactement 4 caractères, censés correspondre à l'année
#resas_info_publicationdate = resas[resas["Date de publication"].astype(str).str.match(r"^\d{4}$")]


resas_info_publicationdate = resas[resas["Date de publication"].astype(str).str.match(r"^.{4}$")]

In [19]:
# LES RESERVATIONS ONT-ELLES ETE RETIREES ?

resas.loc[(resas['waitingdate'].notna())&
          (resas['cancellationdate'].isna()),
          ['Réservation retirée par un adhérent?']] = 'oui'


resas.loc[(resas['waitingdate'].notna())&
          (resas['cancellationdate'].notna()),
          ['Réservation retirée par un adhérent?']] = 'non'

In [20]:
# Les filtres
resas = resas[resas['année de la réservation']<2025] #Résas antérieurs à cette année de la réservation
resas = resas[resas['branchcode']=='MED']

In [21]:
# Evolution du nombre de résas effectuées
resas_evolution_by_year = resas.groupby('année de la réservation')['Nb de réservations effectuées'].sum().to_frame()

In [22]:
# Evolution du nombre de résas effectuées
resas_evolution_by_year = resas.groupby('année de la réservation')['Nb de réservations effectuées'].sum().to_frame()

In [23]:
resas_2024 = resas[resas['année de la réservation']==2024]

resas_2024_by_cards = resas_2024.groupby('categorycode')['Nb de réservations effectuées'].sum().to_frame()

resas_2024_by_cards = resas_2024_by_cards.merge(df_types_cartes,
                                                left_on=resas_2024_by_cards.index,
                                                right_on=df_types_cartes.index,
                                                how='left'
                                               )

filtre_carte = ['carte_gratuite','carte']
resas_2024_by_cards = resas_2024_by_cards.groupby(filtre_carte)['Nb de réservations effectuées'].sum().to_frame()
resas_2024_by_cards

Nb de réservations effectuées
carte_gratuite carte                                                                          
gratuite       Collectivités (Classes maternelles et primaires)                              2
               Collectivités (Structures non scolaires)                                      2
               Consultation sur place                                                       10
               Médiathèque                                                               25462
               Médiathèque Plus (Conservatoire)                                           1378
               Médiathèque Plus (Personnel médiathèque)                                   4931
               Prêt en nombre (imprimés)                                                   199
payante        Médiathèque Plus (17 €)                                                    3219
               Médiathèque Plus (35 €)                                                    3910
               Médiathèque Plus (5 €)                                                     1815

In [24]:
resas_by_week_and_year = pd.pivot_table(data=resas,
                                        index='n° de semaine',
                                        columns='année de la réservation',
                                        values='Nb de réservations effectuées',
                                        aggfunc=sum
                                        )

In [25]:
# Eventuels filtres pour l'année de la réservation 2024
resas_2024[resas_2024['categorycode']!='MEDP']

,reserve_id,borrowernumber,reservedate,biblionumber,branchcode,notificationdate,cancellationdate,priority,found,timestamp,...,waiting_duration,Nb de réservations effectuées,date,vacances_zone_b,nom_vacances,année de la réservation,n° de semaine,jour de la semaine,Date de publication,Réservation retirée par un adhérent?
190640,423845,57877,2024-01-01,349912.0,MED,None,None,0,W,2024-01-02 11:21:25,...,NaN,1,2024-01-01,True,Vacances de Noël,2024,1,NaN,2022,oui
190641,423846,13472,2024-01-01,307294.0,MED,None,None,0,F,2024-01-03 06:18:26,...,0.0,1,2024-01-01,True,Vacances de Noël,2024,1,Mardi,2018,oui
190642,423847,13472,2024-01-01,307294.0,MED,None,2024-01-01,0,None,2024-01-02 03:20:43,...,NaN,1,2024-01-01,True,Vacances de Noël,2024,1,NaN,2018,NaN
190643,423848,13472,2024-01-01,138626.0,MED,None,None,0,F,2024-01-03 06:18:26,...,0.0,1,2024-01-01,True,Vacances de Noël,2024,1,Mardi,2001,oui
190644,423849,13472,2024-01-01,291953.0,MED,None,None,0,F,2024-01-03 06:18:26,...,0.0,1,2024-01-01,True,Vacances de Noël,2024,1,Mardi,2016,oui
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233974,477670,80630,2024-12-31,368666.0,MED,None,None,3,None,2025-01-01 03:13:11,...,NaN,1,2024-12-31,True,Vacances de Noël,2024,1,NaN,2024,NaN
233977,477673,31733,2024-12-31,139076.0,MED,None,None,0,W,2025-01-02 09:32:55,...,NaN,1,2024-12-31,True,Vacances de Noël,2024,1,NaN,2004,oui
233978,477674,18744,2024-12-31,299674.0,MED,None,None,0,F,2025-01-04 08:05:58,...,0.0,1,2024-12-31,True,Vacances de Noël,2024,1,Vendredi,2014,oui
233979,477675,15476,2024-12-31,369397.0,MED,None,None,9,None,2025-01-02 17:41:40,...,NaN,1,2024-12-31,True,Vacances de Noël,2024,1,NaN,2024,NaN


In [26]:
# Rythme des résas dans la semaine selon la période

# Vacances
resas_2024_vacances = resas_2024[resas_2024['vacances_zone_b']==True]
resas_2024_scolaire = resas_2024[resas_2024['vacances_zone_b']==False]

#resas_2024_vacances = resas_2024_docs_empruntés_by_week[resas_2024_docs_empruntés_by_week['vacances_zone_b']==True]
#resas_2024_scolaire = resas_2024_docs_empruntés_by_week[resas_2024_docs_empruntés_by_week['vacances_zone_b']==False]

In [27]:
filter_cols = ['reservedate','waitingdate','cancellationdate']

In [28]:
evolution_resas_retirees = resas.groupby(["année de la réservation","Réservation retirée par un adhérent?"]).size().to_frame('nb de résas').T

In [29]:
evolution_resas_retirees

année de la réservation              2020        2021        2022        2023  \
Réservation retirée par un adhérent?  non    oui  non    oui  non    oui  non   
nb de résas                           546  41243  764  43385  601  36089  466   

année de la réservation                     2024         
Réservation retirée par un adhérent?    oui  non    oui  
nb de résas                           30211  455  31791

In [30]:
resas_2024_by_users = resas_2024.groupby('borrowernumber').size().to_frame('Nb de réservations effectuées')

In [31]:
resas_2024_by_users.loc[resas_2024_by_users['Nb de réservations effectuées']==1,["A réservé en 2024"]] = 'a/ 1 fois' 
resas_2024_by_users.loc[resas_2024_by_users['Nb de réservations effectuées']==2,["A réservé en 2024"]] = 'b/ 2 fois'
resas_2024_by_users.loc[resas_2024_by_users['Nb de réservations effectuées'].between(3,5),["A réservé en 2024"]] = 'c/ Entre 3 et 5 fois'
resas_2024_by_users.loc[resas_2024_by_users['Nb de réservations effectuées'].between(6,10),["A réservé en 2024"]] = 'd/ Entre 6 et 10 fois'
resas_2024_by_users.loc[resas_2024_by_users['Nb de réservations effectuées'].between(10,20),["A réservé en 2024"]] = 'e/ Entre 10 et 20 fois'
resas_2024_by_users.loc[resas_2024_by_users['Nb de réservations effectuées']>20,["A réservé en 2024"]] = 'f/ Plus de 20 fois'

In [32]:
resas_2024_by_users_filtr = resas_2024_by_users.groupby('A réservé en 2024').size().to_frame("Nb d'adhérents")

In [33]:
resas_retirees_2024_bycards = resas_2024.groupby(["Réservation retirée par un adhérent?",'categorycode']).size().to_frame('Nb résas')

In [34]:
#resas_2024.groupby('espace').size().to_frame('Nb de réservations')
resas_2024_by_espace = resas_2024[resas_2024['categorycode']!='MEDP'].groupby('espace').size().to_frame('Nb de réservations')

In [35]:
#resas_2024.groupby('espace')['borrowernumber'].nunique().to_frame('Nb de réserveur.ses distinct.e.s')
reserveurs_distincts_2024_by_espace = resas_2024.groupby('espace')['borrowernumber'].nunique().to_frame('Nb de réserveur.ses distinct.e.s')

In [36]:
resas_2024_by_publicationdate = pd.pivot_table(data=resas_2024,
                                               #data=resas_info_publicationdate[resas_info_publicationdate['année de la réservation']==2024],
                                                index='Date de publication',
                                                values='Nb de réservations effectuées',
                                                aggfunc=sum
                                                )

resas_2024_by_publicationdate

,Nb de réservations effectuées
Date de publication,
,208
0520,1
1880,1
1889,3
1915,2
...,...
2022,3291
2023,5840
2024,8936


# Enregistrement des tableaux

In [37]:
filepath = '../data/projet_tutoré_analyse_resas.xlsx'
with pd.ExcelWriter(filepath) as writer:
    resas_evolution_by_year.to_excel(writer,sheet_name='EvolutionResas')
    resas_2024_by_cards.to_excel(writer,sheet_name='2024NbResasPaTypeDeCarte')
    resas_by_week_and_year.to_excel(writer,sheet_name='NbResasParSemaineParAn')
    evolution_resas_retirees.to_excel(writer,sheet_name='EvolutionResasRetirees')
    resas_2024_by_users_filtr.to_excel(writer,sheet_name='2024FrequenceDesResas')
    resas_retirees_2024_bycards.to_excel(writer,sheet_name='2024NbResasRetireesParTypeCarte')
    resas_2024_by_espace.to_excel(writer,sheet_name='2024NbResasEffectuesParEspace')
    reserveurs_distincts_2024_by_espace.to_excel(writer,sheet_name='2024NbReserveursParEspace')
    resas_2024_by_publicationdate.to_excel(writer,sheet_name='2024ResasParDatePublication')